# N-Dimensional Perceptron

This Jupyter Notebook implements an N-dimensional perceptron. More information can be found in the repository README.

## Setup

### Packages

In [13]:
!pip install pandas numpy matplotlib scikit-learn

You should consider upgrading via the '/home/finn/Source/Conifer/venv/bin/python3.10 -m pip install --upgrade pip' command.


### Importing Packages

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## Data Loading

In [9]:
DATA_DIR = 'data'
DATA_FILE = 'neo_v2.csv'
DATA_FILEPATH = f'{DATA_DIR}/{DATA_FILE}'

In [10]:
data = pd.read_csv(DATA_FILEPATH, header=0)
X = 

### Clean and Format the Data

### Split the Data

In [15]:
X_train, X_test, y_train, y_test = train_test_split(data, random_state=42, test_size=0.15)

ValueError: not enough values to unpack (expected 4, got 2)